In [ ]:
!echo $AWS_ACCESS_KEY_ID

In [2]:
project_prefix = "crypto-cloud-dev-650251698703"
data_lake_bucket_name = "crypto-cloud-dev-650251698703-data-lake-bucket"
data_lake_iceberg_lock_table_name = "crypto_cloud_dev_650251698703_iceberg_lock_table"
data_prefix = project_prefix.replace("-", "_")

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = (
    SparkSession.builder
    .appName("GlueIcebergLocalTest")
    .master("local[*]")
    .config(
        "spark.jars.packages",
        "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1,"
        "org.apache.iceberg:iceberg-aws-bundle:1.6.1,"
        "org.apache.hadoop:hadoop-aws:3.3.4"
    )
    .config("spark.sql.catalog.glue_catalog", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.glue_catalog.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog")
    .config("spark.sql.catalog.glue_catalog.warehouse", f"s3://{data_lake_bucket_name}/")
    .config("spark.sql.catalog.glue_catalog.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.glue_catalog.lock.table", f"{data_lake_iceberg_lock_table_name}")
    .config("spark.sql.defaultCatalog", "glue_catalog")
    .getOrCreate()
)

25/11/04 17:28:19 WARN Utils: Your hostname, Nguyens-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.20 instead (on interface en0)
25/11/04 17:28:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/anhtu/.ivy2/cache
The jars for the packages stored in: /Users/anhtu/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.iceberg#iceberg-aws-bundle added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5b2ddc04-ef54-40b4-9a8c-6886039a0f5a;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/anhtu/.pyenv/versions/3.11.11/envs/crypto-cloud-batch/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 in central
	found org.apache.iceberg#iceberg-aws-bundle;1.6.1 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 112ms :: artifacts dl 5ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.apache.iceberg#iceberg-aws-bundle;1.6.1 from central in [default]
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	-------------------------------

In [5]:
test_db = f"{data_prefix}_iceberg_test_db"
spark.sql(f"""
CREATE DATABASE IF NOT EXISTS {test_db}
LOCATION 's3://{data_lake_bucket_name}/iceberg_test_db/'
""")

DataFrame[]

In [6]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {test_db}.sample_orders (
    order_id INT,
    product STRING,
    price DOUBLE
) USING iceberg
TBLPROPERTIES ('format-version'='2')
""")

DataFrame[]

In [7]:
spark.sql(f"""
INSERT INTO {test_db}.sample_orders
VALUES (3, 'BTC', 62500.1), (4, 'ETH', 3100.5)
""")

DataFrame[]

In [8]:
df = spark.sql(f"SELECT * FROM {test_db}.sample_orders")
df.show()

+--------+-------+-------+
|order_id|product|  price|
+--------+-------+-------+
|       3|    BTC|62500.1|
|       4|    ETH| 3100.5|
|       1|    BTC|65000.1|
|       2|    ETH| 3200.5|
+--------+-------+-------+



In [9]:
spark.stop()